In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
import serpapi

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-;")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
SERPAPI_KEY = os.getenv("SERPAPI_KEY")
print("SERPAPI loaded" if SERPAPI_KEY else "AGGGHHHHH")

SERPAPI loaded


In [4]:
def search_serpapi(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": 5
    }
    results = serpapi.search(params)
    print(results)
    return results


In [5]:
def build_context_from_serpapi(data):
    context = ""
    sources = []

    # Use Organic Results
    for result in data.get("organic_results", []):
        title = result.get("title", "")
        link = result.get("link", "")
        snippet = result.get("snippet", "")
        context += f"Title: {title}\nSnippet: {snippet}\nSource: {link}\n\n"
        sources.append(link)

    # Optionally: Add top stories
    for story in data.get("top_stories", []):
        title = story.get("title", "")
        link = story.get("link", "")
        context += f"Top Story: {title}\nSource: {link}\n\n"
        sources.append(link)

    return context.strip(), sources


In [6]:
openai = OpenAI()
def generate_gpt_answer(query, context):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that uses real-time web information to answer accurately."},
        {"role": "user", "content": f"Question: {query}\n\nContext from web:\n{context}"}
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.5,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()


In [7]:
def ask_web_agent(query):
    data = search_serpapi(query)
    print(data)
    context, sources = build_context_from_serpapi(data)
    final_answer = generate_gpt_answer(query, context)
    return final_answer, sources

# Test the agent
query = "What is the latest update on the Iran-Israel war?"
answer, citations = ask_web_agent(query)
print("Answer:\n", answer)
print("\nSources:")
for src in citations:
    print("-", src)


{
    "search_metadata": {
        "id": "685472be10b5cf656411b366",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/d7d0b2048ca53ad7/685472be10b5cf656411b366.json",
        "created_at": "2025-06-19 20:27:42 UTC",
        "processed_at": "2025-06-19 20:27:42 UTC",
        "google_url": "https://www.google.com/search?q=What+is+the+latest+update+on+the+Iran-Israel+war%3F&oq=What+is+the+latest+update+on+the+Iran-Israel+war%3F&num=5&sourceid=chrome&ie=UTF-8",
        "raw_html_file": "https://serpapi.com/searches/d7d0b2048ca53ad7/685472be10b5cf656411b366.html",
        "total_time_taken": 1.31
    },
    "search_parameters": {
        "engine": "google",
        "q": "What is the latest update on the Iran-Israel war?",
        "google_domain": "google.com",
        "num": "5",
        "device": "desktop"
    },
    "search_information": {
        "query_displayed": "What is the latest update on the Iran-Israel war?",
        "total_results": 445000000,
